In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, TimeoutException, InvalidSessionIdException, NoSuchWindowException
import requests, time, os

driver = webdriver.Chrome()
driver.get("https://accounts.mheducation.com/login?app=connect.mheducation.com&redirectUrl=https:%2F%2Fcaas.mheducation.com%2Fcaas%2Fheclr%2FlaunchConnect")
# Locate the username/email box and type an email
time.sleep(2)

In [7]:
driver.find_element(By.ID, "login-email").send_keys("your_email@example.com")
driver.find_element(By.ID, "login-password").send_keys("your_password")
time.sleep(2)
login_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[@id='login-submit-btn']"))
)
login_button.click()

In [8]:
API_KEY = "Your API key" 
API_URL = "https://api.deepseek.com/v1/chat/completions"

def get_answer(question, answers):
    if answers:  # multiple choice or true/false
        prompt = f"""
        Answer the following question: Choose one or more correct choices as your answer.
        Return only the exact text of the chosen choice(s). If multiple, separate with commas.

        Question: {question}
        Choices: {answers}

        Answer:
        """
    else:  # fill in the blank mode
        prompt = f"""
        Answer the following fill-in-the-blank question. 
        Return only the missing word or phrase with no extra text.

        Question: {question}

        Answer:
        """

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "deepseek-chat",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    data = response.json()

    if "choices" not in data:
        err = data.get("error", {}).get("message", "Unknown error")
        raise RuntimeError(f"API returned no choices. Error: {err}")

    raw_answer = (
        data["choices"][0].get("message", {}).get("content") 
        or data["choices"][0].get("text", "")
    ).strip()
    
    return [a.strip() for a in raw_answer.split(",") if a.strip()]


def escape_xpath_text(text: str) -> str:
    # If text has no single quotes, just wrap in single quotes
    if "'" not in text:
        return f"'{text}'"
    # If it has single quotes, split and use concat()
    parts = text.split("'")
    return "concat(" + ", \"'\", ".join(f"'{p}'" for p in parts) + ")"


try:
    START_URL = driver.current_url
except (InvalidSessionIdException, NoSuchWindowException):
    print("Driver session invalid at startup. Relaunching Chrome...")
    try:
        driver.quit()
    except:
        pass
    from selenium import webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    START_URL = "https://your-start-url-here"  # REPLACE this with your actual quiz link
    driver.get(START_URL)
    time.sleep(5)

output_file = open("quiz_answers.txt", "w", encoding="utf-8")

while True:
    try:
        driver.switch_to.window(driver.window_handles[-1])
        question = driver.find_element(By.CSS_SELECTOR, "div.prompt p").text.strip()
        choices = driver.find_elements(By.CSS_SELECTOR, "span.choiceText")
        
        if question and choices:
            answers = [c.text.strip() for c in choices]
        else:
            answers = []
        
        target_text = get_answer(question, answers)
        
        output_file.write(f"Question: {question}\n")
        output_file.write(f"Answer: {', '.join(target_text)}\n")
        output_file.write("-" * 80 + "\n")
        output_file.flush()
        
        if answers:
            # special case: True/False question
            if set([a.lower() for a in answers]) == {"true", "false"}:
                answer_text = target_text[0].lower()
                if "true" in answer_text:
                    option_label = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//span[contains(@class,'choiceText') and text()='True']"))
                    )
                else:
                    option_label = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//span[contains(@class,'choiceText') and text()='False']"))
                    )
                option_label.click()
            else:
                # regular multiple choice
                for answer_text in target_text:
                    safe_text = escape_xpath_text(answer_text)
                    option_label = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, f"//label[contains(., {safe_text})]"))
                    )
                    option_label.click()
                    time.sleep(1)
        else:
            # fill in the blank
            blank_input = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, "input.fitb-input"))
            )
            blank_input.clear()
            blank_input.send_keys(target_text[0])
        
        # Try confidence button
        try:
            high_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@data-automation-id='confidence-buttons--high_confidence']"))
            )
            high_button.click()
        except:
            print("High button not found, continuing...")
        
        time.sleep(2)
        # Next button
        old_question = question
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'next-button')]"))
        )     
        next_button.click()
        time.sleep(2)
        # Wait for next question to load
        WebDriverWait(driver, 15).until(
            lambda d: d.find_element(By.CSS_SELECTOR, "div.prompt p").text.strip() != old_question
        )
    except (NoSuchElementException, TimeoutException) as e:
        print(f"Exception encountered: {e.__class__.__name__}. Attempting page refresh...")
        try:
            driver.refresh()
            time.sleep(5)
            print("Page refreshed, retrying...")
            continue
        except InvalidSessionIdException:
            # fall through to next block if refresh also fails
            pass

    except InvalidSessionIdException:
        print("Browser session lost. Restarting ChromeDriver...")
        try:
            driver.quit()
        except:
            pass
        try:
            # Relaunch browser and reopen last page
            driver = webdriver.Chrome()
            driver.get(START_URL)
            print("New session started successfully, continuing...")
            time.sleep(5)
            continue
        except Exception as e:
            print(f"Could not restart driver: {e}")
            break

    except Exception as e:
        print(f"Unexpected error: {str(e)}. Refreshing before exit...")
        try:
            driver.refresh()
        except:
            pass
        break

output_file.close()

High button not found, continuing...
Unexpected error: HTTPConnectionPool(host='localhost', port=53139): Max retries exceeded with url: /session/ecda105b07bbe90bb9edee1bd2f17cc5/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1121d86b0>: Failed to establish a new connection: [Errno 61] Connection refused')). Refreshing before exit...
